In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import numpy as np
from collections import defaultdict
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import PyPDF2
import fitz
import re

In [2]:
approvals = list(pd.read_csv('aiml_dfs/aiml_list.csv')['0'].values)

In [3]:
# Feature name mapping for HTML
mapping = {'Device Classification Name': 'device_classification_name',
          '510(k) Number': 'approval_number',
           'De Novo Number': 'approval_number',
          'Device Name': 'device_name',
          'Applicant': 'applicant_name',
           'Requester': 'applicant_name',
          'Applicant Contact': 'applicant_contact',
          'Correspondent': 'correspondent_name',
          'Regulation Number': 'regulation_number',
          'Classification Product Code': 'classification_product_code',
          'Date Received': 'date_received',
          'Decision Date': 'decision_date',
           'Decision': 'decision',
           'Classification Advisory Committee': 'classification_advisory',
           'Review Advisory Committee': 'review_advisory',
           'Regulation Medical Specialty': 'regulation_medical_specialty',
           '510k Review Panel': 'review_panel',
           'statement': 'summary_link',
           'summary': 'summary_link',
           'FDA Review': 'summary_link',
           'Reclassification Order': 'reclass_link',
           'Type': 'type',
           'Reviewed by Third Party': 'reviewed_by_third_party',
           'Combination Product': 'combination_product'
          }

In [19]:
entries = []

for approval in approvals:

    # Parse metadata from FDA website
    with open('FDA_htmls/{}.html'.format(approval), 'r') as fp:
        soup = BeautifulSoup(fp)
    entry = {}
    entry['query_id'] = approval
    for table in soup.findAll('table', {'align': 'center', 'style': 'text-transform: capitalize'}):
        for tr in table.findAll('tr'):
            try:
                th = tr.find('th').text
            except:
                continue

            for k, v in mapping.items():
                if k in th:
                    if v == 'summary_link':
                        link = tr.find('a', href=True)['href']
                        if len(link) == 0:
                            link = tr.find('A', href=True)['HREF']
                        entry[v] = link.strip().lower().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
                    else:
                        text = tr.find('td').text
                        if len(text) > 0:
                            entry[v] = text.strip().lower().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    if len(entry) > 0:                    
        entries.append(entry)
    else:
        raise ValueError('No values found for HTML file {}'.format(approval))
        
df = pd.DataFrame(entries)

In [21]:
df.to_csv('aiml_df.csv', index=False)